<a href="https://colab.research.google.com/github/AgnaldoCC/lab_7_recinfo/blob/master/lab_7_recinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:

import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
resultado = pd.read_csv('https://raw.githubusercontent.com/AgnaldoCC/lab_6_recinfo/master/results.csv')
json = pd.read_json('https://raw.githubusercontent.com/AgnaldoCC/lab_7_recinfo/master/results_final.json')
feedback = {json['query'][i]:json['docs'][i] for i in range(10)}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
tokens = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
M = resultado.text.count()

for i in range(len(resultado)):
  text = resultado.text[i]
  words = [word for word in tokens.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  for h in words:
    if h not in indices.keys():
      indices[h] = []
    indices[h].append(i)
    
for elem in indices.items():
  d = dict(collections.Counter(elem[1]))
  indices[elem[0]] = list(d.items())
  
for word in indices:
  k = len(indices[word])
  IDF = round(np.log((M+1)/k),2)
  indices[word].append(IDF)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # Remove the CWD from sys.path while we load stuff.


**1. Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank**

Foi escolhido o documento 206, que fala sobre uma doação feita a família de Bolsonaro, no valor de 100.000 reais, por 2 acessores.

In [49]:
ndoc = 206
document = resultado.loc[ndoc]
query = 'assessores bolsonaro'

document.title

'Dois assessores de Bolsonaro doaram mais de 100.000 reais a campanhas da fam\xc3\xadlia'

Seguem abaixo as definições de modelos vetoriais feitos no lab anterior.

In [0]:
def tf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    score += doc_tokens.count(word)
  
  return score

In [0]:

def binaria(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  
  for token in query_tokens:
    score += (token in doc_tokens)
    
  return score

In [0]:
def bm25(query, document, k):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  words = [word for word in query_tokens if word in doc_tokens]
    
  for word in words:
    cwd = doc_tokens.count(word)
    dfw = 0
    if word in indices:
      dfw = len(indices[word][:-1])
    score += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(score,2)

In [0]:
def tfidf(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for word in query_tokens:
    cwd = doc_tokens.count(word)
    if word in indices:
      score += cwd * indices[word][-1]
  
  return round(score,2)

In [0]:
def cria_modelos(query, l):
  n = 0
  db = []
  dtf = []
  dtfidf = []
  dbm25 = []
  for doc in resultado.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db, (binaria(query, doc), n))
    bisect.insort(dtf, (tf(query,doc), n))
    bisect.insort(dtfidf, (tfidf(query,doc), n))
    bisect.insort(dbm25, (bm25(query,doc,20), n))
  
  db.reverse()
  dtf.reverse()
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:l], dtf[:l], dtfidf[:l], dbm25[:l]

In [0]:
top_binaria, top_tf, top_tfidf, top_bm25 = cria_modelos(query,10)

**Resultado:**

In [56]:
query_df = pd.DataFrame()

query_df['Binary'] = top_binaria
query_df['TF'] = top_tf
query_df['TF-IDF'] = top_tfidf
query_df['BM25'] = top_bm25

query_df.index+=1
query_df

,Binary,TF,TF-IDF,BM25
1,"(2, 240)","(52, 207)","(111.32, 207)","(22.71, 207)"
2,"(2, 225)","(32, 151)","(35.2, 151)","(5.73, 166)"
3,"(2, 207)","(30, 166)","(33.0, 166)","(5.73, 151)"
4,"(2, 205)","(19, 19)","(20.9, 19)","(4.77, 19)"
5,"(2, 111)","(12, 42)","(13.72, 225)","(3.93, 205)"
6,"(1, 248)","(11, 216)","(13.2, 42)","(3.45, 225)"
7,"(1, 238)","(8, 225)","(12.1, 216)","(3.45, 111)"
8,"(1, 237)","(8, 208)","(11.26, 205)","(3.34, 216)"
9,"(1, 236)","(8, 205)","(10.16, 111)","(3.34, 42)"
10,"(1, 235)","(7, 111)","(8.8, 208)","(2.86, 208)"


**Reciprocal Rank**

In [57]:
def reciprocal_rank(tuplas, id_doc):
  n = 1.0;
  for doc in tuplas:
    if doc == id_doc:
      return  [round(1 / n, 2)]
    else:
      n += 1

rank_df = pd.DataFrame()
rank_df['Binary'] = reciprocal_rank(query_df['Binary'], ndoc)
rank_df['TF'] = reciprocal_rank(query_df['TF'], ndoc)
rank_df['TF-IDF'] = reciprocal_rank(query_df['TF-IDF'], ndoc)
rank_df['BM25'] = reciprocal_rank(query_df['BM25'], ndoc)
rank_df.index+=1
rank_df

,Binary,TF,TF-IDF,BM25
